In [1]:
import pandas as pd 

df=pd.read_csv("df_sucio.csv")
df

,student_id,weekly_self_study_hours,attendance_percentage,class_participation,total_score,grade
0,1,18.5,95.6,3.8,97.9,A
1,2,14.0,80.0,2.5,83.9,B
2,3,19.5,86.3,5.3,100.0,A
3,4,25.7,NaN,7.0,NaN,A
4,5,13.4,81.9,6.9,92.0,A
...,...,...,...,...,...,...
1098787,96493,1.3,62.4,5.2,28.8,F
1098788,509632,16.1,71.6,7.0,93.0,A
1098789,565195,19.4,72.9,9.8,100.0,A
1098790,283407,23.6,97.5,3.0,100.0,A


In [2]:

#cuantos valores faltan por columna 
df.isnull().sum()

student_id                 32963
weekly_self_study_hours    32963
attendance_percentage      32963
class_participation        32963
total_score                32963
grade                      32963
dtype: int64

In [3]:
#ver cuantos duplicados hay 
df.duplicated().sum()

np.int64(61949)

vemos que hay 61949  valores duplicados por lo que procederemos a eliminarlos con"drop_duplicates()" coomo se muestra acontinuacion:

In [ ]:
#eliminar duplicados , pero sin modificar la version original de la base sicia obteniendo asi df2
df2=df.drop_duplicates()

In [5]:
#verificar cuantos duplicados quedan
df2.duplicated().sum()

np.int64(0)

In [30]:
# reiniciar indice 
df2= df2.reset_index(drop=True)


In [7]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036843 entries, 0 to 1036842
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   student_id               1003961 non-null  object 
 1   weekly_self_study_hours  1003950 non-null  object 
 2   attendance_percentage    1003936 non-null  float64
 3   class_participation      1003943 non-null  object 
 4   total_score              1003938 non-null  float64
 5   grade                    1003954 non-null  object 
dtypes: float64(2), object(4)
memory usage: 47.5+ MB


In [ ]:
#vemos el tipo de datos que tiene cada columna y el numero de datos "No nulos", ademas el numero de columna y su nombre 
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1098792 entries, 0 to 1098791
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   student_id               1065829 non-null  object 
 1   weekly_self_study_hours  1065829 non-null  object 
 2   attendance_percentage    1065829 non-null  float64
 3   class_participation      1065829 non-null  object 
 4   total_score              1065829 non-null  float64
 5   grade                    1065829 non-null  object 
dtypes: float64(2), object(4)
memory usage: 50.3+ MB


ahora procederemos a  convertir columnas numericas a tipo float , a excepcion de "student_id" debido a que es un identificador por lo cual se quedara como tipo entero "int"

In [12]:
#aqui convertiremos columnas que sean numericas a float 
# Columnas que deben ser numéricas (con posibles decimales)
float_cols = [
    "weekly_self_study_hours",
    "attendance_percentage",
    "class_participation",
    "total_score"
]

# Limpieza y conversión a tipo float
for c in float_cols:
    df2[c] = (df2[c].astype(str)                      # Convertir todo a texto
                     .str.strip()                      # Eliminar espacios
                     .str.replace(",", ".", regex=False)  # Cambiar comas por puntos
                     .str.replace("%", "", regex=False))  # Eliminar el símbolo %
    
    df2[c] = pd.to_numeric(df2[c], errors="coerce")    # Convertir a número (NaN si falla)

Ahora remplazaremos los valores nulos "NaN" con fillna , esto con el datos de la mediana:

In [13]:
# Reemplazamos los NaN en columnas numéricas por su mediana
for c in float_cols:
    df2[c] = df2[c].fillna(df2[c].median())

Aqui se convertira la columna "student_id" a tipo int ya que es un id , si falla convierte cualquier valor no numerico a NaN, para su posteiror remplazo con 

In [14]:
# Convertir student_id a tipo entero (acepta NaN con Int64)
df2["student_id"] = pd.to_numeric(df2["student_id"], errors="coerce").astype("Int64")

Ahora buscaremos convertir la columna "grade" a tipo category ya que son datos que se pueden categorizar y algunos se puden llegar  a repetir 

In [15]:
# Limpiar y convertir las calificaciones a categoría
df2["grade"] = (df2["grade"]
                    .astype(str)
                    .str.strip()
                    .str.upper()
                    .astype("category"))

In [ ]:
#vemos el tipo de dato de cada columna y el numoero de valores nulos que hay en cada una de ellas 
print("🔹 Tipos de datos finales:")
print(df2.dtypes)

print("\n🔹 Valores faltantes por columna:")
print(df2.isnull().sum())

print("\n🔹 Dimensiones del DataFrame:", df2.shape)

🔹 Tipos de datos finales:
student_id                    Int64
weekly_self_study_hours     float64
attendance_percentage       float64
class_participation         float64
total_score                 float64
grade                      category
dtype: object

🔹 Valores faltantes por columna:
student_id                 0
weekly_self_study_hours    0
attendance_percentage      0
class_participation        0
total_score                0
grade                      0
dtype: int64

🔹 Dimensiones del DataFrame: (1036843, 6)


con el siguiente codigo nos aseguramos que todos los valroes de studen_id sean enteros y que no haya valores faltantes , y si es que los hay se rellenan  con ids nuevos y consecutivos sin modificar los que ya estaban 

In [18]:
# Asegura tipo entero con NaN permitido
df2["student_id"] = pd.to_numeric(df2["student_id"], errors="coerce").astype("Int64")

mask = df2["student_id"].isna()
start = (df2["student_id"].max() or 0) + 1  # siguiente ID libre
df2.loc[mask, "student_id"] = pd.RangeIndex(start, start + mask.sum())
df2["student_id"] = df2["student_id"].astype("Int64")

In [31]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1036843 entries, 0 to 1036842
Data columns (total 6 columns):
 #   Column                   Non-Null Count    Dtype   
---  ------                   --------------    -----   
 0   student_id               1036843 non-null  Int64   
 1   weekly_self_study_hours  1036843 non-null  float64 
 2   attendance_percentage    1036843 non-null  float64 
 3   class_participation      1036843 non-null  float64 
 4   total_score              1036843 non-null  float64 
 5   grade                    1036843 non-null  category
dtypes: Int64(1), category(1), float64(4)
memory usage: 41.5 MB


Aqui veremos los valores unicos de la columan "grade"  normalmente tiene  que aparecer valores NaN  

In [ ]:
df2["grade"].unique(), 

['A', 'B', 'C', 'D', 'F']
Categories (5, object): ['A', 'B', 'C', 'D', 'F']

No se observo un valor nulo pero anteirormente si lo tenia por cual porcederemos a remplazarlos con fillna

In [ ]:
import numpy as np

# 1) Normalizar texto en grade (quitar espacios y poner mayúsculas)
df2["grade"] = df2["grade"].astype(str).str.strip().str.upper()

# 2) Convertir "falsos NaN" en NaN reales
falsos_nan = {"NAN", "NA", "NULL", "NONE", "", "N/A", " - ", "-"}
df2["grade"] = df2["grade"].replace(list(falsos_nan), np.nan)

# 3) (opcional) Limpiar variantes y mapear a A,B,C,D,F si hubiera -> en este caso mi base de datos solo contiene 5 categorias 
mapa = {"A+": "A", "A-":"A", "B+":"B", "B-":"B", "C+":"C", "C-":"C", "D+":"D", "D-":"D",
        "EXCELLENT":"A", "GOOD":"B", "AVERAGE":"C", "POOR":"D", "FAIL":"F"}
df2["grade"] = df2["grade"].replace(mapa)

# 4) Imputar los NaN reales en grade con la moda (valor más frecuente)
moda = df2["grade"].mode(dropna=True)
if len(moda):
    df2["grade"] = df2["grade"].fillna(moda[0])

# 5) Convertir a categoría
df2["grade"] = df2["grade"].astype("category")

# 6) Verificar
print("Únicos en grade:", df2["grade"].unique())
print("NaN en grade:", df2["grade"].isna().sum())
print(df2["grade"].dtype)

Únicos en grade: ['A', 'B', 'C', 'D', 'F']
Categories (5, object): ['A', 'B', 'C', 'D', 'F']
NaN en grade: 0
category


vemos que ahoara ya solo hay 5 categorias , cuando antes solo habia 6 contando NaN 
-----------------------------------------------------------------------------------



Despues veremos si todavia hay duplicados y si es el caso se eliminaran 

In [40]:
df2.duplicated().sum()

np.int64(0)

In [39]:
df2= df2.drop_duplicates(keep="first")

In [41]:
df2.isnull().sum()

student_id                 0
weekly_self_study_hours    0
attendance_percentage      0
class_participation        0
total_score                0
grade                      0
dtype: int64

In [43]:
df2.shape

(1034460, 6)

ver si un mismo id esta repetido con datos distintos.

In [44]:
duplicados_id = df2["student_id"].duplicated().sum()
print("Duplicados por student_id:", duplicados_id)

Duplicados por student_id: 25389


como si hay, vamos a elimirnalos dejando solo un id unico por estudiante 

In [ ]:
df2 = df2.drop_duplicates(subset=["student_id"], keep="first")

# y despues volbemos a verificar 

In [46]:
duplicados_id = df2["student_id"].duplicated().sum()
print("Duplicados por student_id:", duplicados_id)

Duplicados por student_id: 0


Ahora usamos df2.info() para ver cauntos valores de mas tenemos y si es posible seguir reduciendo para tratar de que queden el mismo numero de valores que tenia la bsae orifinal cuando estaba limpia y antes de ensucuiarla.

In [55]:
df2.shape
#como podemos ver hay 9071 datos extras 

(1009071, 6)

vemos que hay aproximandamente 54127 ids de mas  que se conservaron debido a que las filas que no estaban duplicadas y que contenian NaN, los cuales no se eliminaron si no que se remplazaron por valores artificiales con fillna ,  

In [56]:
print("Máximo ID:", df2["student_id"].max())
print("Mínimo ID:", df2["student_id"].min())
print("IDs únicos:", df2["student_id"].nunique())
print("Filas totales:", len(df2))

Máximo ID: 1054127
Mínimo ID: 1
IDs únicos: 1009071
Filas totales: 1009071


posteriormenete se aplico la siguiente linea de codigo para conservar solo las filas originales y nor las que se generaron artificialmemnte 

In [57]:
df2 = df2[df2["student_id"] <= 1000000]

In [58]:
print("Filas finales:", len(df2))
print("Máximo ID:", df2["student_id"].max())
print("IDs únicos:", df2["student_id"].nunique())

Filas finales: 954944
Máximo ID: 1000000
IDs únicos: 954944


Como vemos el dataset se depuro completamente , se eliminaron todos ids artificiales , ya no hay duplicados , 

In [ ]:
df2.shape

(954944, 6)

In [62]:
df2.to_csv("dataset_limpio.csv", index=False, encoding="utf-8")